In [1]:
from pathlib import Path
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


In [2]:
import pandas as pd
import re
import os

file_list_path = "/home/amnkmr/SIP/data_team5/data-team5/output_EOR_overdensity_SURFGEN/output1/Cluster_stat_copy.txt"

with open(file_list_path, 'r') as f:
    file_paths = [line.strip() for line in f if line.strip()]

# Fix the path for each file
def fix_path(fp):
    if '/output1/Cluster_stat_copy' in fp:
        return fp.replace('/output1/Cluster_stat_copy', '/output1/Cluster_stat/Cluster_stat_copy')
    return fp

copy_files = [fix_path(fp) for fp in file_paths if 'copy' in os.path.basename(fp)]

print(f"Found {len(copy_files)} 'copy' files.")

all_dfs = []

for file_path in copy_files:
    try:
        df = pd.read_csv(file_path, 
                         sep=r'\s+', 
                         comment='#', 
                         header=None, 
                         engine='python')
        with open(file_path, 'r') as f:
            for line in f:
                if line.startswith('#rho_th'):
                    columns = line.strip().lstrip('#').split()
                    break
        df.columns = columns

        m = re.search(r'z([0-9.]+)', file_path)
        df['z'] = float(m.group(1)) if m else None

        all_dfs.append(df)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    print(final_df.head())
    final_df.to_csv('all_cluster_stats_copy.csv', index=False)
else:
    print("No dataframes loaded.")

FileNotFoundError: [Errno 2] No such file or directory: '/home/amnkmr/SIP/data_team5/data-team5/output_EOR_overdensity_SURFGEN/output1/Cluster_stat_copy.txt'

In [ ]:
import matplotlib.pyplot as plt

x = final_df['FF']
y1 = final_df['LCS']
y2 = 1 - final_df['LCS']
y3 = final_df['FF'] * final_df['LCS']
y4 = final_df['FF'] * (1 - final_df['LCS'])

plt.figure(figsize=(6, 5))
plt.plot(x, y3, color='red', label='f_largest') 
plt.plot(x, y4, color='blue', label='f_other')   
plt.plot(x, y1, color='green', label='LCS', linestyle='--')
plt.plot(x, y2, color='black', label='1 - LCS', linestyle='-.')

# Add vertical line at percolation transition point
plt.axvline(
    x=0.15, 
    color='purple', 
    linestyle=':', 
    linewidth=2.5,
    label=r'Percolation Transition'
)

# Use linear scale for x-axis
plt.xscale('log')
plt.yscale('log')
plt.xlim([1e-2, 1])
plt.ylim([1e-8, 1.05])
plt.xlabel('FF over dense segment (FF_overdense)')
plt.ylabel('Value')
plt.title('HI Overdense Segment')
plt.axhline(0, color='black', lw=1)
plt.axvline(0, color='black', lw=1)
plt.legend(loc='best', frameon=False)
plt.grid(True, which='major', linestyle='--')
plt.tick_params(axis='both', which='both', direction='in')
plt.tight_layout()
plt.savefig('HI_overdense_segment_plot_linearX.png', dpi=300)
plt.show()